# Json
1. load nested-json data to Postgresql
2. query from Python to Postgresql


In [1]:
import psycopg2 
import psycopg2.extras
import psycopg2.extensions
import pandas as pd         # create dataframes 
import calendar             # convert int to month
import json
from config import config

# Get the config params
params_ = config()
# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)
# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()
conn.autocommit = True

In [2]:
# Create a Table to store JSON data:

def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS nested_tweets;
        CREATE UNLOGGED TABLE nested_tweets (
        ID serial NOT NULL PRIMARY KEY,
     retweeted jsonb );""")
# ID serial NOT NULL PRIMARY KEY

# Send the Schema to PSQL
with conn.cursor() as cursor:
    create_staging_table(cursor)

In [5]:
twts = pd.read_json('data/nested_tweets04.json')
df = twts
df['retweeted'] = df #select retweeted column
df = df.iloc[:,1:]
df.head()

,retweeted
0,"{""retweeted"": [""Nothing_retweeted""]}"
1,"{""retweeted"": [{""created_at"": ""Mon Jul 13 01:2..."
2,"{""retweeted"": [""Nothing_retweeted""]}"
3,"{""retweeted"": [""Nothing_retweeted""]}"
4,"{""retweeted"": [{""created_at"": ""Sat Jul 11 18:5..."


In [10]:
# INSERT INTO [table name] ([columns])
# VALUES ([values])
def fcn(df,table,cur):

    if len(df) > 0:
        df_columns = list(df)
        # create (col1,col2,...)
        columns = ",".join(df_columns)

        # create VALUES('%s', '%s",...) one '%s' per column
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

        #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()

fcn(df,'nested_tweets',cur)


In [12]:
cur.execute("""
SELECT *
FROM nested_tweets LIMIT 4;
""")
cur.fetchall()

[(1, {'retweeted': ['Nothing_retweeted']}),
 (2,
  {'retweeted': [{'id': 1282484990459080705,
     'geo': None,
     'lang': 'fr',
     'text': 'Indigenous rights. LGBTQ. Racism. Climate change. Orange Man Bad. https://t.co/eOouenEsqf',
     'user': {'id': 38885578,
      'url': 'https://t.co/x5G93GBB51',
      'lang': None,
      'name': 'Gary Lamphier',
      'id_str': '38885578',
      'entities': {'url': {'urls': [{'url': 'https://t.co/x5G93GBB51',
          'indices': [0, 23],
          'display_url': 'lamphier.ca',
          'expanded_url': 'http://www.lamphier.ca/'}]},
       'description': {'urls': []}},
      'location': 'Edmonton, Alberta',
      'verified': False,
      'following': None,
      'protected': False,
      'time_zone': None,
      'created_at': 'Sat May 09 16:57:33 +0000 2009',
      'utc_offset': None,
      'description': 'Owner Lamphier Communications. Former biz columnist, lifelong news junkie, sports fan, beer lover, proud hubby, dad & grandad.',
      'ge

In [35]:
cur.execute("""
SELECT id, (jsonb_array_elements(retweeted->'retweeted')->'created_at')::text as trx_id 
FROM nested_tweets LIMIT 7;
""")
cur.fetchall()

[(1, None),
 (2, '"Mon Jul 13 01:20:23 +0000 2020"'),
 (3, None),
 (4, None),
 (5, '"Sat Jul 11 18:55:25 +0000 2020"'),
 (6, '"Sun Jul 12 22:52:00 +0000 2020"'),
 (7, '"Sun Jul 12 19:02:50 +0000 2020"')]

In [32]:
# Find entities
# 'retweeted' : [{'entries' :{'urls':[{'url':'the_html'}]}]
cur.execute("""SELECT id, (jsonb_array_elements(retweeted -> 'retweeted')->'entities')::text as trx_id
FROM nested_tweets LIMIT 7;""")
cur.fetchall()

# Same
# cur.execute("""SELECT id, retweeted->'retweeted'->0->'created_at' as trx_id
# FROM nested_tweets LIMIT 7;""")

[(1, None),
 (2,
  '{"urls": [{"url": "https://t.co/eOouenEsqf", "indices": [66, 89], "display_url": "twitter.com/carmrunco/stat…", "expanded_url": "https://twitter.com/carmrunco/status/1282483761406500865"}], "symbols": [], "hashtags": [], "user_mentions": []}'),
 (3, None),
 (4, None),
 (5,
  '{"urls": [{"url": "https://t.co/WcN9JPCBKG", "indices": [117, 140], "display_url": "twitter.com/i/web/status/1…", "expanded_url": "https://twitter.com/i/web/status/1282025722664517632"}], "symbols": [], "hashtags": [], "user_mentions": []}'),
 (6,
  '{"urls": [{"url": "https://t.co/pazSNK0JM6", "indices": [117, 140], "display_url": "twitter.com/i/web/status/1…", "expanded_url": "https://twitter.com/i/web/status/1282447649183084545"}], "symbols": [], "hashtags": [], "user_mentions": []}'),
 (7,
  '{"urls": [{"url": "https://t.co/6NRHU7Cc5J", "indices": [117, 140], "display_url": "twitter.com/i/web/status/1…", "expanded_url": "https://twitter.com/i/web/status/1282389974667743234"}], "symbols": []

In [36]:
cur.execute("""SELECT id, (jsonb_array_elements(retweeted -> 'retweeted')->'entities'->'urls')::text as trx_id
FROM nested_tweets LIMIT 7;""")
cur.fetchall()

[(1, None),
 (2,
  '[{"url": "https://t.co/eOouenEsqf", "indices": [66, 89], "display_url": "twitter.com/carmrunco/stat…", "expanded_url": "https://twitter.com/carmrunco/status/1282483761406500865"}]'),
 (3, None),
 (4, None),
 (5,
  '[{"url": "https://t.co/WcN9JPCBKG", "indices": [117, 140], "display_url": "twitter.com/i/web/status/1…", "expanded_url": "https://twitter.com/i/web/status/1282025722664517632"}]'),
 (6,
  '[{"url": "https://t.co/pazSNK0JM6", "indices": [117, 140], "display_url": "twitter.com/i/web/status/1…", "expanded_url": "https://twitter.com/i/web/status/1282447649183084545"}]'),
 (7,
  '[{"url": "https://t.co/6NRHU7Cc5J", "indices": [117, 140], "display_url": "twitter.com/i/web/status/1…", "expanded_url": "https://twitter.com/i/web/status/1282389974667743234"}]')]

In [37]:
cur.execute("""SELECT id, (jsonb_array_elements(retweeted->'retweeted')->'entities'->'urls'->0->'url')::text as trx_id
FROM nested_tweets LIMIT 7;""")
cur.fetchall()

[(1, None),
 (2, '"https://t.co/eOouenEsqf"'),
 (3, None),
 (4, None),
 (5, '"https://t.co/WcN9JPCBKG"'),
 (6, '"https://t.co/pazSNK0JM6"'),
 (7, '"https://t.co/6NRHU7Cc5J"')]

In [43]:

cur.execute("""
SELECT id,
(jsonb_array_elements(retweeted->'retweeted')->'entities'->'urls'->0->'url')::text as trx_id
,(jsonb_array_elements(retweeted->'retweeted')->'entities'->'urls'->0->'expanded_url')::text
FROM nested_tweets LIMIT 7;
""")
cur.fetchall()

[(1, None, None),
 (2,
  '"https://t.co/eOouenEsqf"',
  '"https://twitter.com/carmrunco/status/1282483761406500865"'),
 (3, None, None),
 (4, None, None),
 (5,
  '"https://t.co/WcN9JPCBKG"',
  '"https://twitter.com/i/web/status/1282025722664517632"'),
 (6,
  '"https://t.co/pazSNK0JM6"',
  '"https://twitter.com/i/web/status/1282447649183084545"'),
 (7,
  '"https://t.co/6NRHU7Cc5J"',
  '"https://twitter.com/i/web/status/1282389974667743234"')]